In [83]:
from absbox import Generic

seniorBalance = 1000
srtTrancheBal = 200
srtRate = 0.08
closingDate = "2021-06-15"
periodPrincipal = ("curPoolCollection", None, "Principal","Prepayment","Recovery")
reinvestRate = 0.00


srt01 = Generic(
    "SRT Example"
    ,{"cutoff":"2021-06-01","closing":closingDate,"firstPay":"2021-07-20"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":24
          ,"freq":"Monthly","type":"Level","originDate":"2021-06-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":24
          ,"status":"current"}]],
     }
    ,(("acc01",{"balance":0})
      ,("srtAcc",{"balance":0.0
                  ,"interest":{"period":"QuarterEnd"
                               ,"rate":reinvestRate
                               ,"lastSettleDate":closingDate}})
      ,("dummy",{"balance":0.0})
      ,)
    ,(("A1",{"balance":seniorBalance
             ,"rate":0.00
             ,"originBalance":seniorBalance
             ,"originRate":0.00
             ,"startDate":closingDate
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":0.0
             ,"rate":srtRate
             ,"originBalance":0
             ,"originRate":srtRate
             ,"startDate":closingDate
             ,"rateType":{"Fixed":srtRate}
             ,"bondType":{"Sequential":None}
             }))
    ,tuple()
    ,{"amortizing":[
          # pay prorata to senior and SRT tranch
         ['accrueAndPayInt',"dummy",["A1"],{"support":["facility","originator"]}
           ]
         #,["calcInt","B"]
         #,["payInt","dummy",["B"],{"support":["facility","originator"]}]

         ,["calcBondPrin",["A1","B"],{"formula":periodPrincipal}] 

         ,["payPrinWithDue","acc01",["A1"]]         
         ,["payPrinWithDue","srtAcc",["B"]]

     ],
      "closingDay":[
                    ["fundWith","srtAcc","B",{"formula":("const",srtTrancheBal)}]
                   ],
      "endOfCollection":[
          # draw loss amount and pay to originator
          ["liqRepayResidual","srtAcc", "originator", {"formula":("curPoolCollection",None,"Losses")}]
      ],
      "cleanUp":[
          ["payIntResidual","srtAcc","B"]
      ]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,{"originator":{"type" : "Unlimited", "start": closingDate}}
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [84]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL, lang='english', check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.15

In [85]:
r = localAPI.run(srt01
                ,poolAssump=("Pool",("Mortgage", {"CDR":0.01}, None, None, None)
                                   ,None
                                   ,None)
                ,runAssump=[("call",("poolFactor",0.10))]
                ,read=True)

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off